# Homework 3  (Due: 10/31/2018)

COEN 281, Fall 2018  
Professor Marwah

---

The objective of this HW is to implement a Naive Bayes classifier to predict whether a tweet was posted by a Republican or Democrat politician. The training data consist of about 13K tweets collected before the 2006 US presidential elections, There are about an equal number of Republican and Democrat tweets, and the tweets belong to three republican and three democrat twitter accounts. 

To represent each tweet, we will use a commonly used model in natural language processing called 'bag of words' model. A bag of words representation of a document (tweet here) consists of words and their frequencies in the document. The order of words is ignored.  

There four main tasks.
1. Tokenization: Parsing and converting the tweets to tokens. [**This is already done for you**]
2. Feature matrix construction from the training data set
3. Learning Naive Bayes parameters, priors and likelihoods, from the feature matrix.
4. Using the learned NB model to predict the labels of the test data set (about 4K tweets).

## Tokenization
This task consists of converting each tweet into a sequence of "tokens" that can be used as features. Tokens are essentially characters and character sequences obtained after using white space as a separator. A lot these are noise that we want to remove; some are words or other character sequences that are useful features. A python package called *NLTK* (natural language toolkit) contains several tokenizers, including one for tweets. We use that tokenizer; in addition we do the following:
- remove stopwords. These are words that are frequently used in a language but do not carry any semantic information, e.g., the, an , a, this, is, was, etc.
- make all tokens lower case (this is done by the tweet tokenizer)
- removing twitter handles (again, done by the tweet tokenizer)
- remove punctuations, http links

Finally, we "lemmatize" the tokens. That means we convert different forms of a word to a common basic form, so that they can be recognized as the same work. E.g., vote, votes, voted would all be converted to vote; geese would be converted to goose,e tc. (There is a less sophisticated version of lemmatizer called a stemmer which just chops words to convert to the same base work; it doesn't work as well as a lemmatizer and we dont use it here.) There is a good description of the NLTK tokenizer [here](https://berkeley-stat159-f17.github.io/stat159-f17/lectures/11-strings/11-nltk..html).

The output of this part is a cleaned up list of tokens for each tweet. 


In [54]:
import pandas as pd
import string
import numpy as np

import nltk
#
# you may need to run the following
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/venkyubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/venkyubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [56]:
# The data set has two columns - screen_name and text (which is the raw tweet)

## load tweets
tweets = pd.read_csv("tweets_train.csv", na_filter=False)

## screen_namee (accounts)
#  democrat - hillary, time kaine, TheDemocrats
# republicans - trunp, pence, GOP


In [57]:
tweets['screen_name'].unique()

array(['GOP', 'TheDemocrats', 'HillaryClinton', 'timkaine', 'mike_pence',
       'realDonaldTrump'], dtype=object)

In [58]:
tweets.head()

,screen_name,text
0,GOP,RT @GOPconvention: #Oregon votes today. That m...
1,TheDemocrats,RT @DWStweets: The choice for 2016 is clear: W...
2,HillaryClinton,Trump's calling for trillion dollar tax cuts f...
3,HillaryClinton,.@TimKaine's guiding principle: the belief tha...
4,timkaine,Glad the Senate could pass a #THUD / MilCon / ...


In [59]:
tweets.describe()

,screen_name,text
count,13000,13000
unique,6,12982
top,realDonaldTrump,MAKE AMERICA GREAT AGAIN!
freq,2217,4


In [60]:
# add labels
# 1 for D's
# 0 for R's
tweets['label'] = tweets['screen_name'].str.contains('TheDemocrats|HillaryClinton|timkaine', regex=True)
tweets.describe()

,screen_name,text,label
count,13000,13000,13000
unique,6,12982,2
top,realDonaldTrump,MAKE AMERICA GREAT AGAIN!,False
freq,2217,4,6554


The training data has 13K tweets, and each of the two classes have about an equal number of tweets.

Now we will define our tokenizer.

In [61]:
from nltk.stem import WordNetLemmatizer
#
#  Input : dataframe with a column names 'text' which contains raw tweets (one per row)
#  Output: A list of lists of tokens corrsponding to the 'text' column
#
def tokenize_tweets2(tweets):
    """Given a df with tweets in 'text' col, this function return tokens as a list of lists"""

    # apply tokenize to the 'text' coolumn in the tweets df
    tweet_tokenizer = nltk.tokenize.TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles=True)
    tokens = tweets['text'].apply(tweet_tokenizer.tokenize)
    
    # filter
    misc = ['rt', '’', '…', '—', 'u', '”', 'w', '“', '...', '️', 'http', 'https']
    to_remove = nltk.corpus.stopwords.words('english') + list(string.punctuation) + misc
    
    lemmatizer = WordNetLemmatizer()
    
    tokens = [[lemmatizer.lemmatize(token) for token in tw if token not in to_remove] for tw in tokens]      
    return(tokens)

In [62]:
all_tokens = tokenize_tweets2(tweets)
print(len(all_tokens))
all_tokens[:10]

13000


[['#oregon', 'vote', 'today', 'mean', '62', 'day', 'https://t.co/OoH9FVb7QS'],
 ['choice',
  '2016',
  'clear',
  'need',
  'another',
  'democrat',
  'white',
  'house',
  '#demdebate',
  '#wearedemocrats',
  'http://t.co/0n5g0YN46f'],
 ["trump's",
  'calling',
  'trillion',
  'dollar',
  'tax',
  'cut',
  'wall',
  'street',
  'time',
  'pay',
  'fair',
  'share',
  'https://t.co/y8vyESIOES'],
 ['guiding',
  'principle',
  'belief',
  'make',
  'difference',
  'public',
  'service',
  'https://t.co/YopSUeMqOX'],
 ['glad',
  'senate',
  'could',
  'pas',
  '#thud',
  'milcon',
  'vetaffairs',
  'approps',
  'bill',
  'solid',
  'provision',
  'virginia',
  'https://t.co/NxIgRC3hDi'],
 ['exclusive',
  'sits',
  'see',
  'sunday',
  'morning',
  '8:',
  '30a',
  'rtv',
  '6',
  'rtv',
  '6',
  'app'],
 ['chatham',
  'town',
  'council',
  'congress',
  'made',
  'strong',
  'mark',
  'community',
  'proud',
  'work',
  'together',
  'behalf',
  'va'],
 ['thank',
  'new',
  'orleans',
  

The tokenizer can still be improved, but we will go with this. 

Let's find the most common tokens, and we will use all tokens that at least occur 25 times as features.

In [63]:
from collections import Counter

counts = Counter([token for tokens in all_tokens for token in tokens])
print(len(counts))
counts.most_common(20)

23459


[('hillary', 1159),
 ('trump', 1144),
 ('great', 749),
 ('clinton', 720),
 ('today', 709),
 ('make', 581),
 ('donald', 576),
 ('president', 564),
 ('day', 552),
 ('thank', 539),
 ('american', 512),
 ('new', 503),
 ('job', 503),
 ('u', 485),
 ('america', 480),
 ('people', 469),
 ('vote', 451),
 ('state', 442),
 ('get', 420),
 ('year', 415)]

In [64]:
top_words = [k for k in counts.keys() if counts.get(k) > 25]
len(top_words)
#print(top_words[3])


927

top_words are our features.
Now let's construct a feature matrix from these top words

## Feature Martix Construction

Problem 1 (15 points) Compute feature matrix

Now we will extract the features from the training data and construct a feature matrix. The bad news is this matrix can be very large. In our case it is about 13K X 1K, or about 13M x 4 bytes ~ 52M, which will easily fit in the RAM of your laptops, but the training set could have easily been 10x or 100x the current size, and the number of features 10x in which case you would be out of luck. The good news is this matrix is likely to be very sparse. In fact, each tweet is not likely to contain more than 10-20 tokens, so even if this matrix becomes very large, we would be okay if we use a sparse representation.

In a sparse representation, only the non-zero entities and their indices are saved. Scipy provides [several formats](https://docs.scipy.org/doc/scipy-0.18.1/reference/sparse.html) for sparse matrices. In this assignment, it doesn't matter which one you use (in fact, we could have even used a dense matrix). However, since we have to sum along columns (or features), the most suitable one is [csc (or compressed sparse column) format](https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.sparse.csc_matrix.html).

To make it easier to estimate priors and likelihoods, we will construct two feature matrices - one for each for the two classes. For this, first we need to figure out how many data points are in each class.

While setting elements of a csc_matrix you may get a 'SparseEfficiencyWarning'; you can ignore that. 

In [65]:
import math as m
num_feat = len(top_words)

# set this to the correct values
#republicans
nTrainR = tweets.groupby("label").get_group(False).label.count() 
#democrats
nTrainD = tweets.groupby("label").get_group(True).label.count()  


#print(nTrainR)
#print(nTrainD)

# create sparse feature matrix
from scipy.sparse import csc_matrix

rfmat = csc_matrix((nTrainR, num_feat), dtype=int)
dfmat = csc_matrix((nTrainD, num_feat), dtype=int)


#
# populate rfmat and dfmat with the counts of the features
# Remember: all tokens are not features
#
tokensR = tokenize_tweets2(tweets[tweets.label==False])
tokensD = tokenize_tweets2(tweets[tweets.label==True])

# a function that might be useful is <list>.index() 
#republicans
cnt1=0
for word in top_words:
    cnt2=0
    for token in tokensR:
        if word in token:
            rfmat.data=np.append(rfmat.data,token.count(word))
            rfmat.indices=np.append(rfmat.indices,cnt2)
        cnt2 = cnt2 + 1
    rfmat.indptr[cnt1+1]=len(rfmat.data)
    cnt1=cnt1+1

#democrats
cnt1=0
for word in top_words:
    cnt2=0
    for token in tokensD:
        if word in token:
            dfmat.data=np.append(dfmat.data,token.count(word))
            dfmat.indices=np.append(dfmat.indices,cnt2)
        cnt2 = cnt2 + 1
    dfmat.indptr[cnt1+1]=len(dfmat.data)
    cnt1=cnt1+1



In [28]:
print(dfmat)

  (11, 0)	1
  (65, 0)	1
  (74, 0)	1
  (96, 0)	1
  (133, 0)	1
  (166, 0)	1
  (182, 0)	1
  (225, 0)	1
  (236, 0)	1
  (243, 0)	1
  (250, 0)	1
  (302, 0)	1
  (315, 0)	1
  (319, 0)	1
  (337, 0)	1
  (400, 0)	1
  (411, 0)	1
  (418, 0)	1
  (515, 0)	1
  (532, 0)	1
  (535, 0)	1
  (552, 0)	1
  (599, 0)	1
  (602, 0)	1
  (610, 0)	1
  :	:
  (4646, 924)	2
  (1274, 925)	1
  (1382, 925)	1
  (2471, 925)	1
  (2659, 925)	1
  (3232, 925)	1
  (3729, 925)	1
  (4019, 925)	1
  (4873, 925)	1
  (4899, 925)	1
  (5573, 925)	1
  (5857, 925)	1
  (6072, 925)	1
  (1408, 926)	1
  (2290, 926)	1
  (2306, 926)	1
  (2313, 926)	1
  (2735, 926)	1
  (3899, 926)	1
  (4776, 926)	1
  (5192, 926)	1
  (5482, 926)	1
  (5523, 926)	1
  (5682, 926)	1
  (6215, 926)	1


In [66]:
print(rfmat)

  (0, 0)	1
  (31, 0)	1
  (98, 0)	1
  (212, 0)	1
  (236, 0)	1
  (247, 0)	1
  (326, 0)	1
  (401, 0)	1
  (552, 0)	1
  (570, 0)	1
  (605, 0)	1
  (617, 0)	1
  (628, 0)	1
  (646, 0)	1
  (678, 0)	1
  (782, 0)	1
  (789, 0)	1
  (806, 0)	1
  (825, 0)	2
  (870, 0)	1
  (988, 0)	1
  (1010, 0)	1
  (1037, 0)	1
  (1039, 0)	1
  (1152, 0)	1
  :	:
  (4454, 925)	1
  (4532, 925)	1
  (5295, 925)	1
  (5534, 925)	1
  (5721, 925)	1
  (5951, 925)	1
  (6397, 925)	1
  (6447, 925)	1
  (1385, 926)	1
  (1428, 926)	1
  (1578, 926)	1
  (1848, 926)	1
  (1908, 926)	1
  (2872, 926)	1
  (2896, 926)	1
  (2944, 926)	1
  (3470, 926)	1
  (3898, 926)	2
  (4047, 926)	1
  (4304, 926)	1
  (4438, 926)	1
  (5105, 926)	1
  (5160, 926)	1
  (5229, 926)	1
  (5686, 926)	1


## Learning Naive Bayes Model Parameters

Problem 2 (5 points) compute log priors

Problem 3 (30 points) compute log likelihoods using Laplace smoothing

Now we can compute the model parameters, this is, the likelihoods and priors for the two classes. As we discussed in class, since the probabilities can be very small numbers, we will compute log likelihoods and log priors. Aslo use Laplace (aka add one) smoothing.

To sum a matrix column, you can use something like dfmat[:,i].sum()

In [95]:
# compute log priors
import math as m
total_count = tweets["label"].count()
print(total_count)

lPriorR=m.log((nTrainR)/total_count)
lPriorD=m.log((nTrainD)/total_count)

print("Log of priors of Republicans: ",lPriorR)
print("Log of priors of Democrats: ",lPriorD)


# compute log likelihoods
#republicans
featureR = np.array([])

for i in range(0,len(top_words)):
    featureR = np.append(featureR, rfmat[:,i].sum())
fLikelihoodR = (featureR + 1)/(rfmat.sum() + 2) #laplace smoothing, here k=2, 2 values for a feature
fLikelihoodR_Nolaplace = (featureR)/(rfmat.sum())

#print(fLikelihoodR)

logLikelihoodR = np.log(fLikelihoodR) 
#print(logLikelihoodR)

logLikelihoodR_Nolaplace = np.log(fLikelihoodR_Nolaplace) 

#democrats
featureD = np.array([])

for i in range(0,len(top_words)):
    featureD = np.append(featureD, dfmat[:,i].sum())
fLikelihoodD = (featureD + 1)/(dfmat.sum() + 2) #laplace smoothing
fLikelihoodD_Nolaplace = (featureD)/(dfmat.sum())
#print(fLikelihoodD)

logLikelihoodD = np.log(fLikelihoodD) 
#print(logLikelihoodD)

logLikelihoodD_Nolaplace = np.log(fLikelihoodD_Nolaplace) 


13000
Log of priors of Republicans:  -0.6848738071849139
Log of priors of Democrats:  -0.7014895740682907


/home/venkyubuntu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in log
/home/venkyubuntu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in log


## Prediction on Test Set

Now we have a trained Naive Bayes classifier. We will load the test data set and make the predictions. Note: If a token is not a feature, ignore it. 

Problem 4 (5 points) Load test data and tokenize

Problem 5 (30 points) Using the trained NB classifier predict the labels

Problem 6 (5 points) Calculate accuracy, recall, and precision of your predictions


In [82]:
#Prob 4
tweetsT = pd.read_csv("tweets_test.csv", na_filter=False)

tweetsT['label'] = tweetsT['screen_name'].str.contains('TheDemocrats|HillaryClinton|timkaine', regex=True)
tweetsT.describe()

testToken = tokenize_tweets2(tweetsT)
print(testToken[:2])

[['staff', 'hosting', 'office', 'hour', 'across', 'virginia', 'next', 'week', 'answer', 'question', 'find', 'location', 'near', 'https://t.co/nulOEkTOKB'], ['enjoyed', 'r', 'community', 'convo', 'today', 'special', 'thx', 'covenant', 'christian', 'h', 'demotte', 'coming', 'https://…']]


In [92]:
#Prob 5
prediction = np.array([]) #final array where prediction is stored

tempR = 0
tempD = 0
log_pred_prob_R = np.array([])
log_pred_prob_D = np.array([])

for each_tweet in testToken:
    for each_word in each_tweet:
        if each_word in top_words:
            tempR = tempR + logLikelihoodR[top_words.index(each_word)]
            tempD = tempD + logLikelihoodD[top_words.index(each_word)]
    tempR = tempR + lPriorR
    tempD = tempD + lPriorR
    
    log_pred_prob_R = np.append(log_pred_prob_R, tempR)
    log_pred_prob_D = np.append(log_pred_prob_D, tempD)
    
    if(tempR < tempD):
        prediction = np.append(prediction, True)
    elif((tempR > tempD)):
        prediction = np.append(prediction, False)
    else:
        # if log prob for predictions is equal, predicting answer on basis of prior probability
        prediction = np.append(prediction, False)
    
    tempR = 0
    tempD = 0
    
print(prediction)

[ 1.  0.  0. ...,  0.  0.  0.]


In [93]:
# problem 6
#initializing variables for true-positives, true-negatives, false-nagatives and false-positives
TP=0
TN=0
FN=0
FP=0
label = tweetsT["label"]   
for i in range(len(tweetsT)):
    if prediction[i]==True and label[i]==True:
        TP=TP+1
    elif prediction[i]==False and label[i]==False:
        TN=TN+1
    elif prediction[i]==False and label[i]==True:
        FN=FN+1
    elif prediction[i]==True and label[i]==False:
        FP=FP+1
print('Accuracy = ',(TP+TN)/(TP+TN+FN+FP))
print('Recall = ',(TP)/(TP+FN))
print('Precision = ',(TP)/(TP+FP))

Accuracy =  0.8108422522103303
Recall =  0.8109700815956482
Precision =  0.8187643020594966


Problem 7 (5 points) List the features with top ten likelihoods for each of the two classes. What is the likelihood for 'hillary', that is, P(hillary|class)? Is it in the top ten? How important is it in this classification problem? 

Solution:
The likehood for 'hillary' is in the top ten likelihoods for each of the two classes.
It has same values in both the classes. Hence it is not an important deciding factor.
P(feature = 'hillary' | Republican) = -4.1629405295561934
P(feature = 'hillary' | Democrat) = -4.2252911744789374

In [89]:
import operator

likelihoodR = {}
likelihoodD = {}

for i in range(0 , len(top_words)):
    likelihoodR[top_words[i]] = logLikelihoodR[i]
    likelihoodD[top_words[i]] = logLikelihoodD[i]
    
#print(feat_likelihood_R)
slikelihoodR = sorted(likelihoodR.items(), key=operator.itemgetter(1), reverse = True)
print("Republicans: top ten likelihoods\n ")
for i in slikelihoodR[:10]:
    print(i)

slikelihoodD = sorted(likelihoodD.items(), key=operator.itemgetter(1), reverse = True)
print("Democrats: top ten likelihoods\n ")
for i in slikelihoodD[:10]:
    print(i)

Republicans: top ten likelihoods
 
('clinton', -4.1410725429196127)
('hillary', -4.1629405295561934)
('great', -4.1940311166262241)
('thank', -4.439348525434756)
('today', -4.6591344910541199)
('day', -4.7166215819718014)
('new', -4.7166215819718014)
('indiana', -4.7375491020777574)
('job', -4.7807657035775399)
('state', -4.7807657035775399)
Democrats: top ten likelihoods
 
('trump', -3.8088337659282852)
('hillary', -4.2252911744789374)
('donald', -4.3242730080277889)
('president', -4.5698237315919448)
('today', -4.7097088342603799)
('american', -4.7211049689912503)
('make', -4.7472312735834699)
('u', -4.8525917892412966)
('vote', -4.8790834046882727)
('one', -5.050723952061893)


Problem 8 (5 points) How important are the priors in this problem?

Solution: Looking at the prior values, it seems they do not play a significant part in calculating the posteriors. They have almost equal and uniform values.

Extra credit (5 points): Compute the accuracy of the test set without Laplace smoothing and compare with the above.
It seems from the observation that the accuracy does not change very much after remove Laplace function. Only difference is that if Laplace is not used, for features that have not appeared even once in a particular class will throw error/warnings as their sum will be 0 and log(0) is undefined.

In [96]:
# extra credit problem 
prediction = np.array([]) #final array where prediction is stored

tempR = 0
tempD = 0
log_pred_prob_R = np.array([])
log_pred_prob_D = np.array([])

for each_tweet in testToken:
    for each_word in each_tweet:
        if each_word in top_words:
            tempR = tempR + logLikelihoodR_Nolaplace[top_words.index(each_word)]
            tempD = tempD + logLikelihoodD_Nolaplace[top_words.index(each_word)]
    tempR = tempR + lPriorR
    tempD = tempD + lPriorR
    
    log_pred_prob_R = np.append(log_pred_prob_R, tempR)
    log_pred_prob_D = np.append(log_pred_prob_D, tempD)
    
    if(tempR < tempD):
        prediction = np.append(prediction, True)
    elif((tempR > tempD)):
        prediction = np.append(prediction, False)
    else:
        # if log prob for predictions is equal, predicting answer on basis of prior probability
        prediction = np.append(prediction, False)
    
    tempR = 0
    tempD = 0
    
print(prediction)

#initializing variables for true-positives, true-negatives, false-nagatives and false-positives
TP=0
TN=0
FN=0
FP=0
label = tweetsT["label"]    # extracting label column to a list; predictions already in list "result"
for i in range(len(tweetsT)):
    if prediction[i]==True and label[i]==True:
        TP=TP+1
    elif prediction[i]==False and label[i]==False:
        TN=TN+1
    elif prediction[i]==False and label[i]==True:
        FN=FN+1
    elif prediction[i]==True and label[i]==False:
        FP=FP+1
print(TP," ", TN, " ",FN," ",FP)
print('Accuracy = ',(TP+TN)/(TP+TN+FN+FP))
print('Recall = ',(TP)/(TP+FN))
print('Precision = ',(TP)/(TP+FP))

[ 1.  0.  0. ...,  0.  0.  0.]
1784   1693   422   399
Accuracy =  0.8089809213587715
Recall =  0.8087035358114234
Precision =  0.8172240036646816
